## Content

- Content
- Description
- Prepare data
- Models | parameters | time - scores
- Test the best model
- Review

## Description

</br> A client "autotrader", requested a model that can predict price of the car based on featuers.
</br> We are give historical data with feature columns and a price column.
</br> The model pipe line should account for prediction time and time it takes to train model.
</br> Faster is better.

#### libs

In [68]:
RANDOM = 2022
FIRST_CAR_YEAR = 1886 # Benz Patent Motor Car
YEAR = 2022
MASS_PRODUCED_CAR_MAX_HP = 1914 # Rimac Nevera

import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score

import lightgbm as lgb

## Prepare Data

### Load Data

In [2]:
localPath = './datasets/autos.csv'
webPath = '/datasets/autos.csv'

path = ''
if (os.path.exists(localPath)):
  path = localPath
elif (os.path.exists(webPath)):
  path = webPath

df = pd.read_csv(path)

### Irrelevant Data

In [3]:
irrelevant_cols = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']

In [4]:
df = df.drop(irrelevant_cols, axis=1)

### Data Types

In [5]:
df.dtypes

Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
Repaired             object
dtype: object

In [6]:
num_cols = ['Price', 'RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
cat_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

### Numerical Columns Cleaning

In [7]:
display(df[num_cols].describe())

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645
std,4514.158514,90.227958,189.850405,37905.341530,3.726421
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


- 'Price' cannot be 0 -> replace with NaN
- 'RegistrationYear' must be in a range [1886, 2023] -> replace with NaN
- 'Power' must be in a range (0, 1479] -> replace with NaN
- 'Kilometer' ok
- 'RegistrationMonth' ok

In [8]:
df_num = df[num_cols]

In [9]:
df_num['Price'] = df_num['Price'].replace(0, None)

In [10]:
def yearRangeFilter(x):
    if x < FIRST_CAR_YEAR or x > YEAR:
        return None
    return x

In [11]:
df_num['RegistrationYear'] = df_num['RegistrationYear'].apply(lambda x: yearRangeFilter(x))

In [12]:
def powerRangeFilter(x):
    if x <= 0 or x > MASS_PRODUCED_CAR_MAX_HP:
        return None
    return x

In [13]:
df_num['Power'] = df_num['Power'].apply(lambda x: powerRangeFilter(x))

In [14]:
display(df_num.describe())

,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354198.000000,314005.000000,354369.000000,354369.000000
mean,2003.084789,120.971061,128211.172535,5.714645
std,7.536418,62.642107,37905.341530,3.726421
min,1910.000000,1.000000,5000.000000,0.000000
25%,1999.000000,75.000000,125000.000000,3.000000
50%,2003.000000,110.000000,150000.000000,6.000000
75%,2008.000000,150.000000,150000.000000,9.000000
max,2019.000000,1910.000000,150000.000000,12.000000


In [15]:
df[num_cols] = df_num

In [16]:
del df_num

Handle the missing values together.

### Categorical Columns Cleaning

In [17]:
df_cat = df[cat_cols]

In [18]:
for c in df_cat.columns:
    print(c, len(df_cat[c].unique()))

VehicleType 9
Gearbox 3
Model 251
FuelType 8
Brand 40
Repaired 3


In [19]:
display(df_cat['VehicleType'].value_counts())

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

- Data categories are evenly spread out, no logical mistakes in grouping.
- There is no use in 'other' category -> replace 'other' with 'None'

In [20]:
df_cat['VehicleType'] = df_cat['VehicleType'].replace('other', None)

In [21]:
display(df_cat['Gearbox'].value_counts())

manual    268251
auto       66285
Name: Gearbox, dtype: int64

Data categories are evenly spread out, no logical mistakes in grouping.

In [22]:
display(df_cat['FuelType'].value_counts())

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

- Gasoline and petrol are likely the same category -> replace 'gasoline' with 'petrol'
- There is no use in 'other' category -> replace 'other' with 'None'

In [23]:
df_cat['FuelType'] = df_cat['FuelType'].replace('gasoline', 'petrol')

In [24]:
df_cat['FuelType'] = df_cat['FuelType'].replace('other', None)

In [25]:
display(df_cat['Repaired'].value_counts())

no     247161
yes     36054
Name: Repaired, dtype: int64

No mistakes in grouping.

In [26]:
print(sorted(df_cat['Brand'].unique()))

['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen', 'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda', 'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart', 'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant', 'volkswagen', 'volvo']


No apparent mistakes. 

In [27]:
print(df_cat['Model'].unique()[:100])

['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz']


We pray for this data.

In [28]:
df[cat_cols] = df_cat

In [29]:
del df_cat

### Missing Values Analysis

In [30]:
n_rows_with_missing_data = df.isna().any(axis=1).sum()

In [31]:
percentage_of_rows_missing_data = n_rows_with_missing_data / len(df.index) * 100

In [32]:
print('percentage_of_rows_missing_data %.2f' % percentage_of_rows_missing_data)

percentage_of_rows_missing_data 34.62


In [33]:
df_miss = df.isna()

In [34]:
def get_miss_count(row):
    out = 0
    for x in row:
        if x:
            out += 1
    return out

In [35]:
miss_count = df_miss.apply(lambda row: get_miss_count(row), axis=1)

In [36]:
print(miss_count.value_counts())

0    231699
1     62239
2     32086
3     15008
4      6026
5      4290
6      2279
7       723
8        19
dtype: int64


Ways to deal with the missing data:
- Drop it
- Mean/Median
- Most Frequent
- k-NN
- Multivariate Imputation by Chained Equation (MICE)
<br>
<br> My solution preference in order
1. Drop it
2. Drop rows with 2+ missing values - fill with (median & frequent)
3. fill with (median & frequent)

### Missing Values

In [37]:
miss_percent = (len(df.index) - len(df.dropna().index)) / len(df.index) * 100

In [38]:
print('%', 'of missing values', '%.2f' % miss_percent)

% of missing values 34.62


In [39]:
df = df.dropna()

### Categorical Encoding

Ways to encode categorical data, consider the curse of dimensionality
- A few dimensions
  - OHE
- A lot dimensions
  - Label Encoding - works if there is a hierarchy of values
  - Frequency Encoding - problem if two categories have the same # of values
  - Mean Target Encoding - use mean of values associated with those categories

and others...

In [40]:
print(cat_cols)

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']


In [41]:
df_cat = df[cat_cols]

In [42]:
for c in df_cat:
    print(c, len(df_cat[c].unique()))

VehicleType 7
Gearbox 2
Model 248
FuelType 5
Brand 39
Repaired 2


OHE for all, excpet Model - Median Target Encoding (because it will avoid edges)

In [43]:
df_cat = df_cat.join(pd.get_dummies(df_cat['VehicleType']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Gearbox']))
df_cat = df_cat.join(pd.get_dummies(df_cat['FuelType']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Brand']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Repaired']))

In [44]:
models = df_cat['Model'].unique()

In [45]:
models_median_target = {}

In [46]:
for m in models:
    targets = df['Price'][df_cat['Model'] == m]
    median = targets.median()
    models_median_target[m] = median

In [47]:
df_cat['Model_target'] = df_cat['Model'].apply(lambda x: models_median_target[x])

In [48]:
print(df_cat.columns)

Index(['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired',
       'bus', 'convertible', 'coupe', 'sedan', 'small', 'suv', 'wagon', 'auto',
       'manual', 'cng', 'electric', 'hybrid', 'lpg', 'petrol', 'alfa_romeo',
       'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen', 'dacia', 'daewoo',
       'daihatsu', 'fiat', 'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia',
       'lada', 'lancia', 'land_rover', 'mazda', 'mercedes_benz', 'mini',
       'mitsubishi', 'nissan', 'opel', 'peugeot', 'porsche', 'renault',
       'rover', 'saab', 'seat', 'skoda', 'smart', 'subaru', 'suzuki', 'toyota',
       'trabant', 'volkswagen', 'volvo', 'no', 'yes', 'Model_target'],
      dtype='object')


Now we are ready to drop cat_cols and merge it into df, and drop cat_cols from df

In [49]:
df_cat = df_cat.drop(cat_cols, axis=1)

In [50]:
df = df.join(df_cat)

In [51]:
df = df.drop(cat_cols, axis=1)

In [52]:
print(df.shape)

(231699, 61)


### Scaling

In [53]:
df_target = df['Price']

In [54]:
df = df.drop('Price', axis=1)

In [56]:
scaler = MinMaxScaler()
scaler.fit(df)
df[df.columns] = scaler.transform(df)

In [58]:
df.head(3)

,RegistrationYear,Power,Kilometer,RegistrationMonth,bus,convertible,coupe,sedan,small,suv,...,smart,subaru,suzuki,toyota,trabant,volkswagen,volvo,no,yes,Model_target
3,0.842593,0.038968,1.000000,0.500000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.161123
4,0.907407,0.035808,0.586207,0.583333,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.192548
5,0.787037,0.053186,1.000000,0.833333,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.203734


In [59]:
df = df.join(df_target)

### Train Valid Test

In [73]:
df_x = df.drop('Price', axis=1)
df_y = df['Price']

In [74]:
df_x_train_valid, df_x_test, df_y_train_valid, df_y_test = \
    train_test_split(df_x, df_y, test_size=0.2, random_state=RANDOM)

In [75]:
df_x_train, df_x_valid, df_y_train, df_y_valid = \
    train_test_split(df_x_train_valid, df_y_train_valid, test_size=0.25, random_state=RANDOM)

## Models | parameters | time - scores

#### Linear Regression

In [76]:
model = LinearRegression().fit(df_x_train.values, df_y_train.values)

In [77]:
df_a_train = model.predict(df_x_train.values)

In [78]:
r2_score(df_y_train, df_a_train)

0.6603066275692331

0.66 score on training data seems sad

In [79]:
df_a_valid = model.predict(df_x_valid.values)

In [80]:
r2_score(df_y_valid, df_a_valid)

0.6686758802355859

0.69 score on the validation selection is not bad! :)

#### LightGBM

In [99]:
g_x_train = df_x_train
g_y_train = df_y_train
g_x_valid = df_x_valid
g_y_valid = df_y_valid

In [100]:
for c in g_x_train:
    g_x_train[c] = g_x_train[c].astype('float')

g_y_train = g_y_train.astype('float')

for c in g_x_valid:
    g_x_valid[c] = g_x_valid[c].astype('float')

g_y_valid = g_y_valid.astype('float')

In [101]:
lgb_train = lgb.Dataset(g_x_train, g_y_train)
lgb_valid = lgb.Dataset(g_x_valid, g_y_valid, reference=lgb_train)

In [102]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [103]:
# print('Starting training...')
# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=20,
#                 valid_sets=lgb_valid,
#                 callbacks=[lgb.early_stopping(stopping_rounds=5)])

# print('Saving model...')
# gbm.save_model('model.txt')

I couldn't figure out what was causing the problem here!

## Test the best model

## Review

Wanted to share this version, because I need some help, and to make sure that the work I have done so far looks good.